# Problema 3

Considere que en una habitación con MxN espacios, hay P robots limpiadores reactivos. Cada uno de los robot limpiadores se comporta de la siguiente manera:

- Si la celda en la que se encuentra está sucia, entonces aspira durante 10 segundos.
- Si la celda está limpia, el robot elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas que esté sin otro robot) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda). Este movimiento dura 2 segundos.
- Si varios robots coinciden en alguna de las celdas, uno se queda en dicha celda y los otros se tiene que mover a alguna otra celda. Quien se queda o quien se tiene que mover se elige aleatoriamente.
Al inicio de la simulación, las posiciones de los robots son elegidas al azar, y de igual forma las posiciones están limpias o sucias aleatoriamente.

Realiza al menos 5 simulaciones en este entorno con diferente número de robots, y reporta lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias.
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Qué comportamiento agregarías a los robots para que limpiaran con mayor velocidad?

In [316]:
# Model design
import agentpy as ap
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [317]:
# Directions the robot can move to
MOVES = [(1,0),(-1,0),(0,-1),(0,1),(-1,-1),(1,1),(-1,1),(1,-1)]

class VacuumModel(ap.Model):
    
    def setup(self):
        
        # Define the number of trash cells and robots in the simulation
        n_trash = 200
        n_robots = 150
        
        # Generate the agents lists
        self.robot = ap.AgentList(self, n_robots)
        self.trash = ap.AgentList(self, n_trash)
        
        # Grid creatinon
        m = 20
        n = 20
        gridShape = [m, n]
        
        # Defining grid area 
        self.area = ap.Grid(self, gridShape, track_empty=True)   
        
        # Add agents to the environment
        # Agents position generated randomly
        self.area.add_agents(self.robot, random=True) 
        self.area.add_agents(self.trash, random=True, empty=True)

        # Initiate dynamic variables for agents
        # 0: Robot  1: Dirty  2: Clean 
        self.robot.condition = 0
        self.robot.steps = 0
        self.trash.condition = 1
        
        
    def step(self):
        
        robots = self.robot
             
        # Start robot cleaner logic
        for robot in robots:
            
            robot_pos = self.area.positions[robot]
            possible_moves = MOVES.copy()
            for neighbor in self.area.neighbors(robot):
                
                if neighbor.condition == 0 and neighbor.id != robot.id:
                    neighbor_pos = self.area.positions[neighbor]
                    dx = neighbor_pos[1] - robot_pos[1]
                    dy = neighbor_pos[0] - robot_pos[0]
                    target = (dy,dx)
                    if target in possible_moves:
                        possible_moves.remove((dy,dx)) 
                    else:
                        continue
                    
                if neighbor.condition == 1:
                    neighbor.condition = 2
                    break
            else:
                robot.steps += 1
                self.area.move_by(robot, random.choice(possible_moves))
            
        # Stop when there's no trash left
        if len(self.trash.select(self.trash.condition == 1)) == 0:
            self.stop()
                
                    
    def end(self):
        print("uwu")


In [318]:
# Define parameters

parameters = {
    'steps': 1000,
}

In [319]:
# Create single-run animation with custom colors
    
def animation_plot(model, ax):
    attr_grid = model.area.attr_grid('condition')
    color_dict = {0:'#000000', 1:'#a88732', 2:'#d5e5d5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of an area getting cleaned\n"
                 f"Time-step: {model.t}, Dirty area left: {len(model.trash.select(model.trash.condition == 1))}, Robots Moves: {sum(model.robot.steps)}")
    
fig, ax = plt.subplots() 
model = VacuumModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))